# Text Cleaner for Scraped Tweets.

In [35]:
import pandas as pd
from os.path import join, exists
import os
import numpy as np
import re
pd.set_option('max_colwidth', 1000)
from warnings import filterwarnings
filterwarnings('ignore')

root = r'D:\ML_projects\IPV-Scraper\results'
filename = 'df_merged_search_terms_21-Apr-2021_nep.csv'
filepath = join(root, filename)


In [36]:
df = pd.read_csv(filepath, encoding = 'utf-8', skip_blank_lines=True)
df.sample(10)

,date,tweet,link,keyword
417,2022-04-14 19:20:19,अनि आफ्नै सरकारले गलत गरेसी त्यस्को जवाफदेहिता हामी होईनौ है भन्ने कस्ता गतिछाडा नेता यि ?,https://twitter.com/SuhangN/status/1514598175000043524,गतिछाडा
136,2022-04-19 19:53:47,पौडेल काकाले कुल-थर कै इज्जत फाल्लान भन्ने भओ लौन कसैले सम्झाइद्दायो...,https://twitter.com/DpakPoudyal/status/1516418535777574923,इज्जत
706,2022-04-17 07:54:09,"'ओलीजी काम गर्छन्, हामी गठबन्धन मिलाएर सरकारमा आएर भत्काउँछौं' । चरित्र अनुसार मिल्ने नारा चाहिँ यो हुन्छ । ओलीले गरेका कामसँग तुलना गरेर तथ्य दिन सक्नुहुन्छ ?",https://twitter.com/BajagainRam/status/1515512659520933888,चरित्रहीन OR चरित्र
345,2022-04-14 11:05:55,संगै खाउला भनेकै हो कुकुरको जति इज्जत देनउ अब त्यै कुकुर संग बसेर खाउँ,https://twitter.com/SumanSu79899710/status/1514473755698900992,कुकुर OR कुकुर्नी
851,2022-04-21 09:33:23,"दुखको आवरणले मनको खुशी पनि ढाकी दिन्छ तिमीले मलाई बिर्से पनि म हरपल सम्झिरहन्छु आशा जिउँदै छ,तिमीले बुझ्ने कोशिस गर किनकी तिमीले न नबुझे मेरो माया मर्न सक्छ।",https://twitter.com/susan_poudyal/status/1516987181625225221,जिउदै OR जिउँदै
1278,2022-04-20 13:17:46,घरमा आलु को सब्जी बनाउदा गाउँ भरी सुन्ने गरि म खान्न मोटाईन्छ भन्ने पापकी परि क्याफे मा तारेको आलु Tasty भन्दै ठुस्दि रैछे। boyfriend संग मेलो लाजा मेलो बुला भन्दै मस्किदै बसेर्,https://twitter.com/Syangjali_kale/status/1516681265319055361,मस्किदै
770,2022-04-15 16:05:06,त्यो काठमाण्डुको छाडा गाई गोरुलाई गिद्ध रेस्टुरेन्टमा लगेर छोडे हुने नि ।,https://twitter.com/saradritu1/status/1514911433162690565,छाडा
399,2020-11-07 08:36:19,खातिनी कुकुर्नी नै रैछे नि,https://twitter.com/blinkbibash/status/1324907252700995584,खातिनी OR खातिनि
233,2022-04-19 20:06:57,"काङ्रेस र माउवादी त एकै भएछन्, कुकुर बिरालो जस्तो दुस्मनी त तिम्रो र मेरो मात्र भो प्रिय",https://twitter.com/Nirmaladhakal44/status/1516421851127422984,कुकुर OR कुकुर्नी
435,2021-06-21 20:09:23,नेपालमा हुँदो हो त यो ईमेलले घर भाँड्ने रहेछ।,https://twitter.com/sinja2060/status/1406981326515826701,घर भाँड्ने


In [51]:
pattern1 = r'(_[a-zA-Z0-9]+)|(#[\u0900-\u097F]+)|(@[\u0900-\u097F]+)|(_[\u0900-\u097F]+)'
to_replace = """@#=/+…:"")(}{][*%_’‘'"""
pattern2 = r'(\W)(?=\1)'

## Text Cleaner function.
### Sequential steps:
        - Remove all the words starting with '_' ("_ahiraj"), mentions starting with '_' ("@_Silent__Eyes__") and also Devanagiri ("@पाेखरा") and hashtags used with Devanagiri ("#पाेखरा").
        - Remove punctuations (selected manually). This does not include sentence enders like "|" or "." or "?" or "!".
        - Removes bad characters like "&gt;".
        - If a punctuation or a whitespace has been repeated multiple times, adjust it to a single occurence.


In [52]:
def cleaner(text: str) -> str:
    """
    Cleans the tweets using the fllowing sequential steps:
        - Remove all the words starting with '_' ("_ahiraj"), mentions starting with '_' ("@_Silent__Eyes__") and also Devanagiri ("@पाेखरा") and hashtags used with Devanagiri ("#पाेखरा").
        - Remove punctuations (selected manually). This does not include sentence enders like "|" or "." or "?" or "!".
        - Removes bad characters like "&gt;".
        - If a punctuation or a whitespace has been repeated multiple times, adjust it to a single occurence.

    Args:
        text (str): Input text.

    Returns:
        str: Cleaned text.
    """    
    text = re.sub(pattern1, '', text)
    text = text.translate(str.maketrans('', '', to_replace))
    text = text.translate(str.maketrans('', '', '&gt;'))
    text = re.sub(pattern2, '', text)
    return text

## Apply text cleaner function to the tweets.

In [53]:
df['tweet'] = df['tweet'].apply(cleaner)

In [66]:
df['tweet'].sample(20)

1241                                                                                                                                                                                                                        भैरबस्थान मन्दिर नजिकै ७० वर्षीया महिलालाई बलात्कार गरेपछि.
271                                                                             सहि कुरो दिदिको! कुकुर पाले भन्दैमा आफुलाई कुल सम्झिने नै हुन् दूध को भाउ बढ्दा बिरोध गर्ने। त्यस्ता स्वाठा लाई एक लबटा हान्नुपर्ने। -राम्रै कुकुर डोर्याएको छ कुना, काप्चा, झाडिमा कुर्क्याउन लग्या छ।
1413                                                                                                                                                                                         धेरै ताती सकिस यादगर एकदिन आफै सितल हुन्छेस सुन बुढो खोज्दै हिड्छेस एकदिन आफै पितल हुन्छेस
985                                                                                                                                 ए धर्मका नाममा कुसंस्कार पाल

## Save to a CSV file.

In [67]:
df.to_csv(join(root, 'scraped_tweets_cleaned_22-04-022.csv'), index = None, encoding = 'utf-8')